In [1]:
boundingBox = []
centroid = []
fileName = []

In [2]:
import cv2
from PIL import Image, ImageEnhance
import numpy as np

In [17]:
def separateChars(originalFile):
    
    #increase brightness
    im = Image.open("data/" + originalFile)
    
    #WARNING: No checks on abnormal dimensions done yet!
    new_width  = 300
    new_height = 300
    im = im.resize((new_width, new_height), Image.ANTIALIAS)
    
    enhancer = ImageEnhance.Brightness(im)
    enhanced_im = enhancer.enhance(1.5)
    im.save("data/bright.png")

    # read the image
    image = cv2.imread("data/bright.png")
    origWidth, origHeight, channels = image.shape

    # apply greyscale filter onto image
    image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # apply a blur to remove noise from the image
    #image_grey = cv2.GaussianBlur(image_grey, (7, 7), 0)

    # threshold image to inverted binary - black and white only
    ret, thresh1 = cv2.threshold(image_grey ,127,255,cv2.THRESH_BINARY_INV)
    
    # dilate image to make white portions thicker
    image_dilate = cv2.dilate(thresh1, None, iterations=2)
    
    # greyscale image from now on provides much better identification
    # all cropped bounding boxes will be from the greyscale image
    image = image_grey
    
    cv2.imwrite("separated/PROCESSED.png",image_dilate)

    # find ALL contours in the image
    contours = cv2.findContours(image_dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    original = image.copy()
    i = 0

    for contour in contours:
        print("AREA: " + str(cv2.contourArea(contour)))
        # if contour area is too small, ignore it
        if(cv2.contourArea(contour) < 900): continue

        # bounding rectangles placed on all contours detected
        x,y,w,h = cv2.boundingRect(contour)
        #
        if w*h < 25: continue

        # region of interest taken from contour - we are interested in the contents of the rectangle only
        roi = image[y:y+h, x:x+w]
        #roi = image[y:y+45, x:x+45]

        boundingBox.insert(i, [x,y,w,h])

        centroidX = (x+w)/2
        centroidY = (y+h)/2

        centroid.insert(i, [centroidX,centroidY])
    
        fileName.insert(i, "roi"+str(i)+".png")
        
        # save the contents of each rectangle as a separate PNG image
        cv2.imwrite("separated/"+fileName[i], roi)
        
        
        #resize h and w of roi
        imgi = Image.open("separated/"+fileName[i])
    
        base_roi_width  = 25
        #new_roi_height = 45
        wpercent = (base_roi_width / float(imgi.size[0]))
        hsize = int((float(imgi.size[1]) * float(wpercent)))
        imgi = imgi.resize((base_roi_width, hsize), Image.ANTIALIAS)
        #imgi = imgi.resize((new_roi_width, new_roi_height), Image.ANTIALIAS)
    
        #imgi.save("separated/"+fileName[i])
        
        
        # read image
        img = cv2.imread("separated/"+fileName[i])
        ht, wd, cc= img.shape

        # create new image of desired size and color for padding
        ww = 150                                             
        hh = 150
        color = (255,255,255)
        result = np.full((hh,ww,cc), color, dtype=np.uint8)

        # compute center offset
        xx = (ww - wd) // 2
        yy = (hh - ht) // 2
 
        # copy img image into center of result image
        result[yy:yy+ht, xx:xx+wd] = img

        # save result
        cv2.imwrite("separated/"+fileName[i], result)
    
        i = i + 1
        
    return i

In [18]:
separateChars("D.png")

AREA: 1282.5
AREA: 1724.5


2

In [11]:
def getBoundingBoxes():
    return boundingBox

In [60]:
def getCentroids():
    return centroid

In [61]:
def getFilenames():
    return fileName